## Projeto 2


### Como microdados que descrevem uma pessoa diagnosticada com um câncer específico afetam sua taxa de mortalidade.


---------

#### Lendo a base de dados


In [ ]:
import pandas
from  IPython.display import display
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
import statistics as stats
import re